In [32]:
# Library import

import warnings

warnings.filterwarnings('ignore')
warnings.simplefilter('ignore')

import pandas as pd
from pathlib import Path
import datetime as dt
import numpy as np
import os
import matplotlib.pyplot as plt
import seaborn as sns

from google.cloud import bigquery
from google.oauth2 import service_account

delta=dt.timedelta(days=1)

In [33]:
# Get main path and other folders
s_mainpath=Path.cwd()
print(s_mainpath)
s_parentpath=s_mainpath.parent.parent.parent
print(s_parentpath)
# folderpath=s_parentpath.joinpath(r'DatosXM\Despacho\\')

# Ruta General
sRutaP=r'C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\\'

sRutaPrint=s_mainpath.joinpath('Print')

c:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython
c:\Alejo


In [34]:
# Lectura de datos de enrtrada para el cálculo
s_parentpath=Path('C:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython\ArchivosEntrada')
filepath=s_parentpath.joinpath(s_parentpath,'BarrasDigLP.xlsx')

# Carga del nivel probabilístico del embalse
sheet_name='Barra_Fkey'
df_MapBF=pd.read_excel(filepath, header=0,sheet_name=sheet_name)
# Df_MapBF=Df_MapBF.set_index('Embalse_Sinergox')
# df_MapBF

In [35]:
# List all files in the folder
file_names = [] 
l_years = []
l_tipo=['Max','Med','Min']

for f in os.listdir(sRutaP):

    if os.path.isfile(os.path.join(sRutaP, f)):

        year=int(f[0:4])
        file_names.append(f)
        l_years.append(year)
    
file_names.sort()
l_years = list(set(l_years))
l_years.sort()

s_FileExcel='C:\Alejo\Eje de Planeación\Análisis Eléctrico\ScriptsPython\DememandaLP.xlsx'
with pd.ExcelWriter(s_FileExcel,engine='xlsxwriter') as writerExcel:

    for year in l_years:

        df_AuxF=pd.DataFrame()
        iban=0

        for tipo in l_tipo:
    
            l_colGen=['Fkey','Estado_' + tipo,'P_' + tipo,'Q_' + tipo,'Col5']
            file=str(year) + '_' + tipo + '.dgs'
            filepath=Path(sRutaP + file)
            print(filepath)
            df_Aux = pd.read_csv(filepath, sep=';',names=l_colGen,encoding="ISO-8859-1")

            df_Aux=df_Aux.drop(['Col5'],axis=1)
            df_Aux = df_Aux.iloc[3:]
            # Split the column by ',' and remove blank spaces
            df_Aux['Fkey'] = df_Aux['Fkey'].str[2:].str.strip()

            df_Aux['P_' + tipo] = df_Aux['P_' + tipo].astype(float)
            df_Aux['P_' + tipo] = df_Aux['P_' + tipo].round(3)

            df_Aux['Q_' + tipo] = df_Aux['Q_' + tipo].astype(float)
            df_Aux['Q_' + tipo] = df_Aux['Q_' + tipo].round(3)

            df_Aux['Estado_' + tipo] = df_Aux['Estado_' + tipo].astype(float)
            df_Aux['Estado_' + tipo] = df_Aux['Estado_' + tipo].astype(int)

            l_colGen=l_colGen[:-1]
            l_colGen.insert(0, 'Barra')
            df_Aux=df_Aux.merge(df_MapBF,left_on=['Fkey'],right_on=['Fkey'], how='left')[l_colGen]

            if iban==0:
                df_AuxF=pd.concat([df_AuxF,df_Aux],axis=0)
                iban=1
            else:
                l_col = list(df_AuxF.columns)
                l_col = l_col + l_colGen[2:]
                df_AuxF=df_AuxF.merge(df_Aux,left_on=['Barra','Fkey'],right_on=['Barra','Fkey'], how='left')[l_col]

        # Exportar a excel la información de cada fecha con el detalle
        s_sheetname=str(year)
        df_AuxF.to_excel(writerExcel, sheet_name=s_sheetname, index=False)
        worksheet = writerExcel.sheets[s_sheetname]
        worksheet.set_column('A:A', 20)
        worksheet.set_column('B:B', 15)


C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2022_Max.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2022_Med.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2022_Min.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2023_Max.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2023_Med.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2023_Min.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2024_Max.dgs
C:\Información XM\Publico\PlaneacionOperacion\LargoPlazo\IPOELP\2024\Semestre02\Proyección Demanda\Nacional\2024_Med.dgs
C:\Información XM\Publico\Planea